In [ ]:
import sys, time, os, asyncio
import scipy.stats as stats
from scipy.signal import find_peaks
from scipy import signal
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time
from matplotlib.ticker import FormatStrFormatter
from lsst_efd_client import EfdClient
from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient, clipDataToEvent, calcNextDay
%matplotlib inline
%load_ext autoreload
%autoreload 

In [ ]:
# Functions to get data
key_m1m3_dict={'1 X': 'm1m3_x_1', 
            '1 Y': 'm1m3_y_1', 
            '1 Z': 'm1m3_z_1', 
            '2 X': 'm1m3_x_2', 
            '2 Y': 'm1m3_z_2', # note these two have been 
            '2 Z': 'm1m3_y_2', # switched pending SUMMIT-7911
            '3 X': 'm1m3_x_3', 
            '3 Y': 'm1m3_y_3', 
            '3 Z': 'm1m3_z_3'
            }
key_m2_dict={'1 X': 'm2_x_1', 
            '1 Y': 'm2_y_1', 
            '1 Z': 'm2_z_1', 
            '2 X': 'm2_x_2', 
            '2 Y': 'm2_z_2', 
            '2 Z': 'm2_y_2', 
            '3 X': 'm2_x_3', 
            '3 Y': 'm2_z_3', 
            '3 Z': 'm2_y_3', 
            '4 X': 'm2_x_4', 
            '4 Y': 'm2_y_4', 
            '4 Z': 'm2_z_4', 
            '5 X': 'm2_x_5', 
            '5 Y': 'm2_z_5', 
            '5 Z': 'm2_y_5',
            '6 X': 'm2_x_6', 
            '6 Y': 'm2_z_6', 
            '6 Z': 'm2_y_6', 
            }
def vms_data_to_pandas(filename, vms_type, begin_time=None, end_time=None):
    """
    Converts VMS data in the given HDF5 file to a Pandas DataFrame.

    Args:
    filename: Path to the HDF5 file containing the VMS data.
    vms_type: The type of VMS data in the file. Must be "m1m3", "m2", or
      "rotator".
    begin_time: The start time of the data to include in the DataFrame. If None,
      all data will be included.
    end_time: The end time of the data to include in the DataFrame. If None, all
      data will be included.

    Returns:
    A Pandas DataFrame containing the VMS data.
    """
    if vms_type == "m1m3":
        key_dict=key_m1m3_dict
    elif vms_type=="m2":
        key_dict=key_m2_dict
    elif vms_type=="rotator":
        raise NotImplementedError
    else:
        raise ValueError('vms_type must be m1m3,m2, or rotator')

    f = h5py.File(filename, 'r')
    times = f['timestamp'][::1]
    dkeys = 'XYZ'
   
    data_dict = {}
    if (begin_time is not None) & (end_time is not None): 
        sel = (times > begin_time) & (times < end_time)
    else: 
        sel = np.ones(times.size).astype(bool)
    data_dict['times'] = times[sel]  
    for key in key_dict.keys():
        data_dict[key_dict[key]] = f[key][::1][sel]
    data_frame = pd.DataFrame(data_dict)
    for j in np.arange(int(len(key_dict)/3)) +1:
        data_frame[f"total_{j}"] = np.linalg.norm(
            data_frame[[f"{vms_type}_{i}_{j}" for i in ["x","y","z"]]].values, axis=1
        )
    
    
    return data_frame

def get_freq_psd(vals, timestep):

    """
    Calculates the frequency power spectrum of a signal.

    Args:
        vals (np.array): The signal values.
        timestep (float): The time step between samples.

    Returns:
        tuple: The frequencies and power spectral density.
    """

    # Remove the mean from the signal.

    meanval = np.mean(vals)
    signal = vals - meanval

    # Calculate the length of the signal.

    N = len(signal)

    # Calculate the power spectral density.

    psd = np.abs(np.fft.rfft(np.array(signal) * 1)) ** 2

    # Calculate the frequencies.

    frequencies = np.fft.rfftfreq(N, timestep)
    return (frequencies, psd)



def get_peak_points(freq, psd, height=0.01):
    """
    Get the peak points of the power spectral density (PSD).

    Args:
        freq (numpy.ndarray): The frequency vector.
        psd (numpy.ndarray): The power spectral density.
        height (float): The minimum peak height.

    Returns:
        numpy.ndarray: The peak points.
    """

    # Find the peak indices and heights.
    peak_ind, peak_dict = find_peaks(psd, height=height)
    peaks = freq[peak_ind]

    # If there are no peaks, return None.
    if len(peaks) < 1:
        return None

    # Find the sub-peaks within each group of peaks that are close in frequency.
    points = []
    for i, peak in enumerate(peaks):
        sel = (abs(peaks - peak) < 1)
        sub_peaks = peaks[sel]
        sub_heights = peak_dict['peak_heights'][sel]
        points.append(sub_peaks[np.argmax(sub_heights)])

    # Return the unique peak points.
    return np.unique(np.array(points))

if you can please check M1M3 VMS data local time (UTC-3) 11:20-12:30 and 14:40-15:00 - when we run the FCUs

In [ ]:
table="lsst.sal.MTM1M3.accelerometerData"
begin_time=Time('2023-12-07 14:40:00.00')
end_time=Time('2023-12-07 14:40:05.00')
client = EfdClient("usdf_efd")
vms_m1m3_filename="/home/p/pferguso/u/scratch/vms_data/12/M1M3-2023-12-07T00:00.hdf"

In [ ]:
vms_m1m3_data=vms_data_to_pandas(vms_m1m3_filename, vms_type="m1m3",begin_time=begin_time.unix, end_time=end_time.unix)

In [ ]:
fig, axs = plt.subplots(3,1, sharex=True, figsize=(12,5), dpi=175)
accel_smooth=50
vms_times=Time(vms_m1m3_data["times"], format="unix").datetime
for i in range(3):
    ax=axs[i]
    ax.plot(vms_times, vms_m1m3_data[f"total_{i+1}"].rolling(50).mean(), label=i+1)
    
    ax.set_yticks(np.arange(0.001,0.002,0.001))
plt.subplots_adjust(hspace=0)
axs[1].set_ylabel("VMS total acceleration\nChannel 2")
axs[0].set_ylabel("Channel 1")
axs[2].set_ylabel("Channel 3")
ax.set_xlabel("Time")
fig.suptitle(f"M1M3 FCU #1\n{begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}\n Total Vibration", y=0.99)


In [ ]:
def get_psd_and_dsd_for_vms(vals,timestep,min_freq=1, g=False):
    # assume vms data is in m/s^2
    if g:
        #assume units are milig
        vals = 1e-3 * 9.8 * vals
    freq,accel_psd=get_freq_psd(vals, timestep)
    # remove 0 to avoid divide by zero
    sel = (freq > min_freq)
    freq = freq[sel]
    accel_psd = accel_psd[sel]
    # returned psd is in units of m/s^2/(1/timestep) convert to m/s^2/hz
    accel_psd = accel_psd * np.mean(np.diff(freq))

    disp_psd_sq = accel_psd / ((2 * np.pi * freq)**4)
    
    int_displace_psd = np.sqrt(np.cumsum(disp_psd_sq))
    
    total_displacement = np.sqrt(np.sum(disp_psd_sq))

    psds_df = pd.DataFrame({"freq": freq, "accel_psd": accel_psd, 
                         "disp_psd": np.sqrt(disp_psd_sq),
                         "int_disp": int_displace_psd,
                        })
    
    return psds_df, total_displacement

In [ ]:
begin_time_on=Time('2023-12-07 14:33:00.00')
end_time_on=Time('2023-12-07 14:50:00.00')
vms_m1m3_data_on=vms_data_to_pandas(vms_m1m3_filename, vms_type="m1m3",begin_time=begin_time_on.unix, end_time=end_time_on.unix)
vms_m1m3_data_on["total_1"][vms_m1m3_data_on["total_1"] > 0.0035] = np.mean(vms_m1m3_data_on["total_1"])

begin_time_off=Time('2023-12-07 14:20:00.00')
end_time_off=Time('2023-12-07 14:30:00.00')
vms_m1m3_data_off=vms_data_to_pandas(vms_m1m3_filename, vms_type="m1m3",begin_time=begin_time_off.unix, end_time=end_time_off.unix)

In [ ]:
fig, axs = plt.subplots(3,1, sharex=True, figsize=(12,5), dpi=175)

vms_times_on=Time(vms_m1m3_data_on["times"], format="unix").datetime
vms_times_off=Time(vms_m1m3_data_off["times"], format="unix").datetime
for i in range(3):
    ax=axs[i]
    ax.plot(vms_times_on, vms_m1m3_data_on[f"total_{i+1}"], label=f"FCUs on")
    ax.plot(vms_times_off, vms_m1m3_data_off[f"total_{i+1}"], label=f"FCUs off")
    
    ax.set_yticks(np.arange(0.000,0.005,0.001))
    ax.set_ylim(-0.001,0.005)
plt.subplots_adjust(hspace=0)
axs[0].legend()
axs[1].set_ylabel("VMS total acceleration [m/s^2]\nChannel 2")
axs[0].set_ylabel("Channel 1")
axs[2].set_ylabel("Channel 3")
ax.set_xlabel("Time")
fig.suptitle(f"M1M3 FCU #1\n{begin_time.iso[:10]} {begin_time_off.iso[11:19]}-{end_time_on.iso[11:19]}\n Total Vibration", y=0.99)


In [ ]:
@Petr from https://ts-xml.lsst.io/sal_interfaces/MTVMS.html#telemetry the VMS accelerations in the efd are in m/s^2, is this true for the VMS data stored in hdf files as well?

In [ ]:
print(f"Assuming units of VMS = mg")
for key in [f"total_{i+1}" for i in range(3)]:
    _, total_disp_on =get_psd_and_dsd_for_vms(vms_m1m3_data_on[key], 
                                                   np.mean(np.diff(vms_m1m3_data_on["times"])),
                                                   g=True, min_freq=1)
    _, total_disp_off =get_psd_and_dsd_for_vms(vms_m1m3_data_off[key], 
                                                     np.mean(np.diff(vms_m1m3_data_on["times"])),
                                                     g=True, min_freq=1)
    print(f"-displacement for {key}\n FCU on = {total_disp_on * 1e6:0.2f}" + " \u03BCm" + f"\n FCU off = {total_disp_off * 1e6:0.2f}" + " \u03BCm"  )

print()
print(f"Assuming units of VMS = m/s^2")
for key in [f"total_{i+1}" for i in range(3)]:
    _, total_disp_on =get_psd_and_dsd_for_vms(vms_m1m3_data_on[key], 
                                                   np.mean(np.diff(vms_m1m3_data_on["times"])),
                                                   g=False, min_freq=1)
    _, total_disp_off =get_psd_and_dsd_for_vms(vms_m1m3_data_off[key], 
                                                     np.mean(np.diff(vms_m1m3_data_on["times"])),
                                                     g=False, min_freq=1)
    print(f"-displacement for {key}\n FCU on = {total_disp_on * 1e6:0.2f}" + " \u03BCm" + f"\n FCU off = {total_disp_off * 1e6:0.2f}" + " \u03BCm"  )

In [ ]:
fig, axs = plt.subplots(3,1,dpi=125,figsize=(10,10), sharex=True)
#plt.suptitle(f"M1M3 FCU #1\n{begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}\n PSD for all axes of all channels", y=0.99)

key="total_1"
for i,key in enumerate([f"total_{i+1}" for i in range(3)]):
    ax=axs[i]
    psds_df_on, total_disp_on =get_psd_and_dsd_for_vms(vms_m1m3_data_on[key], 
                                                       np.mean(np.diff(vms_m1m3_data_on["times"])),
                                                       g=True, min_freq=1)
    psds_df_off, total_disp_off =get_psd_and_dsd_for_vms(vms_m1m3_data_off[key], 
                                                         np.mean(np.diff(vms_m1m3_data_on["times"])),
                                                         g=True, min_freq=1)
    ax.plot(psds_df_on['freq'], np.sqrt(psds_df_on['accel_psd']) , label="FCU" + " on", zorder=9, lw=1)
    ax.plot(psds_df_off['freq'], np.sqrt(psds_df_off['accel_psd']) , label="FCU"  + " off", zorder=9, lw=1)
    title_str= f"total displacement, on: FCU on = {total_disp_on * 1e6:0.2f} $\mu m$"
    title_str+=f" FCU off = {total_disp_off * 1e6:0.2f} $\mu m$"
    ax.set(ylabel=f"{key}\npsd [(m/$s^2$)/hz]", title=title_str )
    ax2=ax.twinx()
    ax2.plot(psds_df_on['freq'], psds_df_on['int_disp'] * 1e6 , label="cumulative displacement on", zorder=9, lw=1, ls="dashed")
    ax2.plot(psds_df_off['freq'], psds_df_off['int_disp'] * 1e6 , label="cumulative displacement off", zorder=9, lw=1, ls="dashed")
    ax2.set(ylabel="Cumulative displacement [$\mu$m]")
    if i==0:
        ax2.legend(edgecolor="None", loc="center right")
        ax.legend(edgecolor="None", loc="center left")
# step=30
# ax=axs[0]
# ylimval=(1e-3, 1e2)
# for i,key in enumerate([i for i in sorted(vms_m1m3_data_on.keys()) if ("total" in i)][:1]):
    
#     freq,psd=get_freq_psd(vms_m1m3_data_on[key], np.mean(np.diff(vms_m1m3_data_on["times"])))
#     sel=(freq >=1) #& (freq < 43)
#     freq = freq[sel]
#     psd = psd[sel]
#     disp_psd_sq = psd #/ ((2 * np.pi * freq)**4)
#     int_psd = np.cumsum(disp_psd_sq) * np.mean(np.diff(freq))
#     points_x=get_peak_points(freq, psd, height=1)
#     if points_x is not None:
#         points_y=np.ones_like(points_x) * i * step * -1
#         print(f"{key}: {str([int(i) for i in points_x])[1:-1]} Hz")
#     else: 
#         print(f"{key} no peaks")
#     ax.plot(freq, int_psd , label=key + "on", zorder=9, lw=1)
#     axs[1].plot(freq, psd , label=key + "on", zorder=9, lw=1)
#     axs[1].axhline(1, c="k", ls="dashed")
#    ax.set(yscale='linear')#, ylim= ylimval)
# ax=axs[1]
# for i,key in enumerate([i for i in sorted(vms_m1m3_data_on.keys()) if ("total" in i)][:1]):
    
#     freq,psd=get_freq_psd(vms_m1m3_data_off[key], np.mean(np.diff(vms_m1m3_data_off["times"])))
#     points_x=get_peak_points(freq, psd, height=1)
#     if points_x is not None:
#         points_y=np.ones_like(points_x) * i * step * -1
#         print(f"{key}: {str([int(i) for i in points_x])[1:-1]} Hz")
#     else: 
#         print(f"{key} no peaks")
#     ax.plot(freq,psd - i * step, label=key + "off", zorder=9, lw=1)
#     ax.set(yscale='log', ylim= ylimval)
#plt.legend(ncol=3, edgecolor="k", facecolor="white", loc=9, framealpha=1)
#plt.ylim(-0.05 - i * step, step)

axs[2].set_xlabel("frequency [Hz]")
plt.savefig("psd_comp.png")
# plt.ylabel("psd + offset")
# plt.grid(visible=True, axis="x", ls="dashed", alpha=0.4)
# plt.subplots_adjust(hspace=0)
# _=plt.xticks(np.arange(0,100,10))

In [ ]:
fig, axs = plt.subplots(2,1,dpi=125,figsize=(10,4), sharex=True)
#plt.suptitle(f"M1M3 FCU #1\n{begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}\n PSD for all axes of all channels", y=0.99)
step=30
ax=axs[0]
ylimval=(1e-3, 1e2)
for i,key in enumerate([i for i in sorted(vms_m1m3_data_on.keys()) if ("total" in i)][:1]):
    
    freq,psd=get_freq_psd(vms_m1m3_data_on[key], np.mean(np.diff(vms_m1m3_data_on["times"])))
    sel=(freq >=1) #& (freq < 43)
    freq = freq[sel]
    psd = psd[sel]
    disp_psd_sq = (psd * 100) # to mm/s^2
    disp_psd_sq = disp_psd_sq/ ((2 * np.pi * freq)**4)
    int_psd = np.cumsum(disp_psd_sq * np.mean(np.diff(freq)) ) 
    points_x=get_peak_points(freq, psd, height=1)
    if points_x is not None:
        points_y=np.ones_like(points_x) * i * step * -1
        print(f"{key}: {str([int(i) for i in points_x])[1:-1]} Hz")
    else: 
        print(f"{key} no peaks")
    ax.plot(freq, int_psd , label=key + "on", zorder=9, lw=1)
    axs[1].plot(freq, disp_psd_sq , label=key + "on", zorder=9, lw=1)
    axs[1].axhline(1, c="k", ls="dashed")
    axs[1].set(yscale = 'log')
    ax.set(yscale='linear')#, ylim= ylimval)
    print(np.sqrt(np.sum(disp_psd_sq[:]) * np.mean(np.diff(freq))))
# ax=axs[1]
# for i,key in enumerate([i for i in sorted(vms_m1m3_data_on.keys()) if ("total" in i)][:1]):
    
#     freq,psd=get_freq_psd(vms_m1m3_data_off[key], np.mean(np.diff(vms_m1m3_data_off["times"])))
#     points_x=get_peak_points(freq, psd, height=1)
#     if points_x is not None:
#         points_y=np.ones_like(points_x) * i * step * -1
#         print(f"{key}: {str([int(i) for i in points_x])[1:-1]} Hz")
#     else: 
#         print(f"{key} no peaks")
#     ax.plot(freq,psd - i * step, label=key + "off", zorder=9, lw=1)
#     ax.set(yscale='log', ylim= ylimval)
#plt.legend(ncol=3, edgecolor="k", facecolor="white", loc=9, framealpha=1)
#plt.ylim(-0.05 - i * step, step)
plt.xlabel("frequency [Hz]")
plt.ylabel("psd + offset")
plt.grid(visible=True, axis="x", ls="dashed", alpha=0.4)
plt.subplots_adjust(hspace=0)
_=plt.xticks(np.arange(0,100,10))

In [ ]:
len_val=np.min([vms_m1m3_data_on.shape[0], vms_m1m3_data_off.shape[0]])

In [ ]:
plt.figure(dpi=175,figsize=(10,5))
plt.suptitle(f"M1M3 FCU #1\n{begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}\n PSD for all axes of all channels", y=0.99)
step=30
for i,key in enumerate([i for i in sorted(vms_m1m3_data.keys()) if ("total" in i)]):
    
    freq,psd=get_freq_psd(vms_m1m3_data[key], np.mean(np.diff(vms_m1m3_data["times"])))
    points_x=get_peak_points(freq, psd, height=1)
    if points_x is not None:
        points_y=np.ones_like(points_x) * i * step * -1
        print(f"{key}: {str([int(i) for i in points_x])[1:-1]} Hz")
    else: 
        print(f"{key} no peaks")
    plt.plot(freq,psd - i * step, label=key, zorder=9, lw=1)

plt.legend(ncol=3, edgecolor="k", facecolor="white", loc=9, framealpha=1)
plt.ylim(-0.05 - i * step, step)
plt.xlabel("frequency [Hz]")
plt.ylabel("psd + offset")
plt.grid(visible=True, axis="x", ls="dashed", alpha=0.4)
_=plt.xticks(np.arange(0,110,10))


# Event 2
17:40-18:00


In [ ]:
begin_time=Time('2023-12-07 17:30:00.00')
end_time=Time('2023-12-07 18:50:00.00')
vms_m1m3_data=vms_data_to_pandas(vms_m1m3_filename, vms_type="m1m3",begin_time=begin_time.unix, end_time=end_time.unix)

In [ ]:
fig, axs = plt.subplots(3,1, sharex=True, figsize=(12,5), dpi=175)
accel_smooth=50
vms_times=Time(vms_m1m3_data["times"], format="unix").datetime
for i in range(3):
    ax=axs[i]
    ax.plot(vms_times, vms_m1m3_data[f"total_{i+1}"].rolling(50).mean(), label=i+1)
    
    ax.set_yticks(np.arange(0.001,0.005,0.001))
plt.subplots_adjust(hspace=0)
axs[1].set_ylabel("VMS total acceleration\nChannel 2")
axs[0].set_ylabel("Channel 1")
axs[2].set_ylabel("Channel 3")
ax.set_xlabel("Time")
fig.suptitle(f"M1M3 FCU #2\n{begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}\n Total Vibration", y=0.99)


In [ ]:
begin_time=Time('2023-12-07 17:35:00.00')
end_time=Time('2023-12-07 18:30:00.00')
vms_m1m3_data=vms_data_to_pandas(vms_m1m3_filename, vms_type="m1m3",begin_time=begin_time.unix, end_time=end_time.unix)

In [ ]:
plt.figure(dpi=175,figsize=(10,5))
plt.suptitle(f"M1M3 FCU #2\n{begin_time.iso[:10]} {begin_time.iso[11:19]}-{end_time.iso[11:19]}\n PSD for all axes of all channels", y=0.99)
step=70
for i,key in enumerate([i for i in sorted(vms_m1m3_data.keys()) if ("total" in i)]):
    
    freq,psd=get_freq_psd(vms_m1m3_data[key], np.mean(np.diff(vms_m1m3_data["times"])))
    if i == 1:
        height=1
    else:
        height=6
    points_x=get_peak_points(freq, psd, height=height)
    if points_x is not None:
        points_y=np.ones_like(points_x) * i * step * -1
        print(f"{key}: {str(sorted(np.unique([int(i) for i in points_x])))[1:-1]} Hz")
    else: 
        print(f"{key} no peaks")
    plt.plot(freq,psd - i * step, label=key, zorder=9, lw=1)

plt.legend(ncol=3, edgecolor="k", facecolor="white", loc=9, framealpha=1)
plt.ylim(-0.05 - i * step, step)
plt.xlabel("frequency [Hz]")
plt.ylabel("psd + offset")
plt.grid(visible=True, axis="x", ls="dashed", alpha=0.4)
_=plt.xticks(np.arange(0,110,10))
